# Sophisticated Inference Agent

In [1]:
import sys
import os
from datetime import datetime

# Add the parent directory to sys.path so Python can find the sibling directories
sys.path.insert(0, os.path.abspath('..'))

# Importing standard library
import numpy as np
import random
import matplotlib.pyplot as plt
from datetime import datetime
from contextlib import redirect_stdout
import logging

# Importing local modules
from agent_utils import initialise_distributions
from math_utils import round_half_up
from sophisticated_inference import agent_loop

In [2]:
# Set True to visualise the agent's behaviour using Matplotlib
VISUALISE = False
ax = None
if VISUALISE:
    # Plotting commands should open in a new window
    %matplotlib qt
    fig, ax = plt.subplots()
    plt.ion()  # Turn on interactive mode

![](../figures/Agent_GenerativeModelComponents.png)

### Complete Recipe for Active Inference 
_Taken from PyMDP Active Inference Tutorial_
1. Sample an observation $o_t$ from the current state of the environment.
2. Perform inference over hidden states i.e., optimize $q(s)$ through free-energy minimization.
3. Calculate expected free energy of actions $\mathbf{G}$.
4. Sample action from the posterior over actions $Q(u_t) \sim \sigma(-\mathbf{G})$.
5. Use the sampled action $a_t$ to perturb the generative process and go back to step 1.

In [3]:
# Experimental/environmental hyperparams
num_trials = 99
num_states = 100
num_factors = 2
num_contextual_states = 4
contextual_food_locations = [70,42,56,77]
contextual_water_locations = [72,32,47,66]
contextual_sleep_locations = [63,43,48,58] 
hill_1 = 54 

# Agentic hyperparams
num_modalities = 3
num_resource_observations = 4 # [none, food, water, sleep]
num_context_observations = 5 # [summer, autumn, winter, spring, none]
t_constraint = 100
resource_constraints = {"Food":21,"Water":19,"Sleep":24}
start_position = 50

# Tree search hyperparams
weights = {"Novelty":10, "Learning":40, "Epistemic":1, "Preference":10}
G_prior = 0.02

In [4]:
# Distributions
A, a, B, b, D = initialise_distributions(
    num_states,
    contextual_food_locations,
    contextual_water_locations,
    contextual_sleep_locations,
    hill_1,
    start_position)

Let's check the shapes of the key distributions used in sophisticated inference. 

In [5]:
print(f"Shapes of Variables:")
print(f"{'Variable':<20}{'Shape':<20}")
print(f"{'-'*20}{'-'*20}")
print(f"{'B[0]':<20}{str(B[0].shape):<20}")
print(f"{'B[1]':<20}{str(B[1].shape):<20}")
print(f"{'b[0]':<20}{str(b[0].shape):<20}")
print(f"{'b[1]':<20}{str(b[1].shape):<20}")
#print(f"{'chosen_action':<20}{str(chosen_action.shape):<20}")
print(f"{'a[0]':<20}{str(a[0].shape):<20}")
print(f"{'a[1]':<20}{str(a[1].shape):<20}")
print(f"{'a[2]':<20}{str(a[2].shape):<20}")
print(f"{'A[0]':<20}{str(A[0].shape):<20}")
print(f"{'A[1]':<20}{str(A[1].shape):<20}")
print(f"{'A[2]':<20}{str(A[2].shape):<20}")
print(f"{'D[1]':<20}{str(D[0].shape):<20}")
print(f"{'D[2]':<20}{str(D[1].shape):<20}")
#print(f"{'short_term_memory':<20}{str(short_term_memory.shape):<20}")

Shapes of Variables:
Variable            Shape               
----------------------------------------
B[0]                (100, 100, 5)       
B[1]                (4, 4, 5)           
b[0]                (100, 100, 5)       
b[1]                (4, 4, 5)           
a[0]                (100, 100, 4)       
a[1]                (4, 100, 4)         
a[2]                (5, 100, 4)         
A[0]                (100, 100, 4)       
A[1]                (4, 100, 4)         
A[2]                (5, 100, 4)         
D[1]                (100, 1)            
D[2]                (4, 1)              


## Core Agent Logic

1. **Initialize Resource Timers and Time Variable:**
   - Set `time_since_resource` for "Food," "Water," and "Sleep" to 0.
   - Initialize the time counter `t` to 0.

2. **Begin Main Loop:**
   - Continue if `t` is less than `t_constraint` and the agent is still "alive" based on resource constraints.

3. **Update Environment and Model:**
   - Call `update_environment` to modify the environment and agent model based on the selected action, updating historical records of the agent's posterior beliefs and true states.

4. **Retrieve Current Beliefs:**
   - Get the current belief state `Q` from `historical_agent_posterior_Q`.

5. **Update Agent's Needs:**
   - Call `update_needs` to modify the time since the agent last had each resource, based on the locations of resources and the agent's current state.

6. **Observe Environment:**
   - Call `get_observations` to receive the current observations from the environment based on the agent's true state.

7. **Backward Smoothing:**
   - If the current time `t` is greater than 6, adjust the `start` time for smoothing.
   - Perform backward smoothing from the `start` time to the current time `t`.

8. **Check Belief Accuracy:**
   - Print the length of the `historical_agent_posterior_Q` and its first element for debugging.
   - Calculate the state prediction error and the agent's current position.
   - If the prediction error is substantial or the agent is in a specific position, reset the agent's short-term memory.

9. **Determine Planning Horizon:**
   - Calculate the horizon for the tree search, considering the time constraints from the agent's needs.
   - Ensure the horizon is at least 1.

10. **Perform Tree Search:**
    - Initialize the list of best actions.
    - Execute the forward tree search to determine the best actions, considering the agent's beliefs, observations, and needs.

11. **Record Chosen Action:**
    - Append the chosen action to the historical record.

12. **(Placeholder) Display Results:**
    - A placeholder for future code to display or process the results of this iteration.

13. **Prepare for Next Iteration:**
    - Increment the time counter `t` for the next loop iteration.

In [6]:
# Generate a timestamp in the format YYYY-MM-DD_HH-MM-SS
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Configure logging to write to both the console and a log file
log_file_path = f"../../logs/experiment_log_{timestamp}.txt"
log_format = '%(message)s'  # Only display the log message
logging.basicConfig(
    level=logging.INFO,
    format=log_format,
    handlers=[
        logging.FileHandler(log_file_path, encoding='utf-8'),
        logging.StreamHandler()
    ]
)

In [7]:

# Experiments
trials_data = []
total_start_time = datetime.now()
total_hill_resets = 0
total_pe_resets = 0
total_memory_accessed = 0
total_search_depth = 0
t_at25 = 0
t_at50 = 0
t_at75 = 0
t_at100 = 0
total_t = 0

for trial in range(num_trials):
    logging.info(f"\n{'-' * 40}\nTRIAL {trial+1} STARTED\n{'-' * 40}")
    trial_start_time = datetime.now()
    logging.info(f"Start Time: {trial_start_time.strftime('%Y-%m-%d %H:%M:%S')}")

    if VISUALISE and ax is not None: ax.clear()
    a, trial_data = agent_loop(
        a, 
        resource_constraints, 
        A, 
        B, 
        b, 
        D,
        t_constraint, 
        contextual_food_locations, 
        contextual_sleep_locations, 
        contextual_water_locations, 
        num_modalities, 
        num_states,
        num_factors,             
        num_contextual_states,   
        num_resource_observations,  
        num_context_observations,   
        hill_1,             
        weights,          
        G_prior,
        start_position,     
        visualise = VISUALISE,     
        ax = ax
    )
    trials_data.append(trial_data)

    if trial_data["time_steps"] >= 24 and trial_data["time_steps"] < 49:
        t_at25 = t_at25 + 1
    elif trial_data["time_steps"] >= 49 and trial_data["time_steps"] < 74:
        t_at50 = t_at50 + 1
    elif trial_data["time_steps"] >= 74 and trial_data["time_steps"] < 99:
        t_at75 = t_at75 + 1
    elif trial_data["time_steps"] == 99:
        t_at100 = t_at100 + 1

    total_t += trial_data["time_steps"]-1
    total_hill_resets += trial_data["hill_memory_resets"]
    total_pe_resets += trial_data["pe_memory_resets"]
    total_search_depth += trial_data["search_depth"]
    total_memory_accessed += trial_data["memory_accessed"]
    
    # Calculate total runtime for the latest trial
    trial_end_time = datetime.now()    
    runtime = trial_end_time - trial_start_time
    minutes, seconds = divmod(runtime.seconds, 60)
    
    logging.info(f"TRIAL {trial+1} COMPLETE ✔")
    logging.info(f"End Time: {trial_end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    logging.info(f"Total runtime for this trial (minutes/seconds): {minutes:02d}:{seconds:02d}")
    logging.info(f"{'-' * 40}")
    logging.info(f"Total hill visits: {total_hill_resets}")
    logging.info(f"Total prediction errors: {total_pe_resets}")
    logging.info(f"Total search depth: {total_search_depth}")
    logging.info(f"Total times memory accessed: {total_memory_accessed}")
    logging.info(f"Total times 24 >= t < 49: {t_at25}")
    logging.info(f"Total times 49 >= t < 74: {t_at50}")
    logging.info(f"Total times 74 >= t < 99: {t_at75}")
    logging.info(f"Total times t == 99: {t_at100}")
    logging.info(f"Total time steps survived: {total_t}")
    # Calculate total run time so far
    runtime = trial_end_time - total_start_time
    hours, remainder = divmod(runtime.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    logging.info(f"Total runtime so far (hours/minutes/seconds): {hours:02d}:{minutes:02d}:{seconds:02d}")
    logging.info(f"{'-' * 40}")

# Calculate total run time for all trials
total_end_time = datetime.now()
runtime = total_end_time - total_start_time
hours, remainder = divmod(runtime.seconds, 3600)
minutes, seconds = divmod(remainder, 60)
logging.info(f"EXPERIMENT COMPLETE ✔.")
logging.info(f"End Time: {total_end_time.strftime('%Y-%m-%d %H:%M:%S')}")
logging.info(f"TOTAL RUNTIME (hours/minutes/seconds): {hours:02d}:{minutes:02d}:{seconds:02d}")
logging.info(f"AVERAGE RUNTIME PER TIME STEP: {round_half_up((runtime.total_seconds() / total_t), 3)} seconds")
logging.info(f"Average hill visits per time step: {round_half_up(total_hill_resets / total_t,3)}")
logging.info(f"Average prediction errors per time step: {round_half_up(total_pe_resets / total_t,3)}")
logging.info(f"Average search depth per time step: {round_half_up(total_search_depth / total_t)}")
logging.info(f"Average times memory accessed per time step: {round_half_up(total_memory_accessed / total_t)}")
logging.info(f"{'-' * 40}")


----------------------------------------
TRIAL 1 STARTED
----------------------------------------
Start Time: 2023-11-28 12:59:36


At time 20 the agent is dead.
The agent had: 1 food, -1 water, and 4 sleep.
The total tree search depth for this trial was 1112.
The agent accessed its memory 1754 times.
The agent cleared its short-term memory 3 times.
    State prediction error memory resets: 2.
    Hill memory resets: 1.


TRIAL 1 COMPLETE ✔
End Time: 2023-11-28 12:59:37
Total runtime for this trial (minutes/seconds): 00:01
----------------------------------------
Total hill visits: 1
Total prediction errors: 2
Total search depth: 1112
Total times memory accessed: 1754
Total times 24 >= t < 49: 0
Total times 49 >= t < 74: 0
Total times 74 >= t < 99: 0
Total times t == 99: 0
Total time steps survived: 20
Total runtime so far (hours/minutes/seconds): 00:00:01
----------------------------------------

----------------------------------------
TRIAL 2 STARTED
----------------------------------------
Start Time: 2023-11-28 12:59:37


At time 20 the agent is dead.
The agent had: 1 food, -1 water, and 4 sleep.
The total tree search depth for this trial was 1606.
The agent accessed its memory 3442 times.
The agent cleared its short-term memory 7 times.
    State prediction error memory resets: 7.
    Hill memory resets: 0.


TRIAL 2 COMPLETE ✔
End Time: 2023-11-28 12:59:40
Total runtime for this trial (minutes/seconds): 00:02
----------------------------------------
Total hill visits: 1
Total prediction errors: 9
Total search depth: 2718
Total times memory accessed: 5196
Total times 24 >= t < 49: 0
Total times 49 >= t < 74: 0
Total times 74 >= t < 99: 0
Total times t == 99: 0
Total time steps survived: 40
Total runtime so far (hours/minutes/seconds): 00:00:03
----------------------------------------

----------------------------------------
TRIAL 3 STARTED
----------------------------------------
Start Time: 2023-11-28 12:59:40


...

TRIAL 28 COMPLETE ✔
End Time: 2023-11-28 13:09:01
Total runtime for this trial (minutes/seconds): 01:40
----------------------------------------
Total hill visits: 26
Total prediction errors: 153
Total search depth: 218135
Total times memory accessed: 789054
Total times 24 >= t < 49: 5
Total times 49 >= t < 74: 0
Total times 74 >= t < 99: 0
Total times t == 99: 0
Total time steps survived: 605
Total runtime so far (hours/minutes/seconds): 00:09:25
----------------------------------------

----------------------------------------
TRIAL 29 STARTED
----------------------------------------
Start Time: 2023-11-28 13:09:01


At time 20 the agent is dead.
The agent had: 1 food, -1 water, and 4 sleep.
The total tree search depth for this trial was 3507.
The agent accessed its memory 12136 times.
The agent cleared its short-term memory 10 times.
    State prediction error memory resets: 9.
    Hill memory resets: 1.


TRIAL 29 COMPLETE ✔
End Time: 2023-11-28 13:09:10
Total runtime for this trial (minutes/seconds): 00:08
----------------------------------------
Total hill visits: 27
Total prediction errors: 162
Total search depth: 221642
Total times memory accessed: 801190
Total times 24 >= t < 49: 5
Total times 49 >= t < 74: 0
Total times 74 >= t < 99: 0
Total times t == 99: 0
Total time steps survived: 625
Total runtime so far (hours/minutes/seconds): 00:09:33
----------------------------------------

----------------------------------------
TRIAL 30 STARTED
----------------------------------------
Start Time: 2023-11-28 13:09:10


At time 20 the agent is dead.
The agent had: 1 food, -1 water, and 4 sleep.
The total tree search depth for this trial was 2215.
The agent accessed its memory 6247 times.
The agent cleared its short-term memory 3 times.
    State prediction error memory resets: 2.
    Hill memory resets: 1.


TRIAL 30 COMPLETE ✔
End Time: 2023-11-28 13:09:14
Total runtime for this trial (minutes/seconds): 00:04
----------------------------------------
Total hill visits: 28
Total prediction errors: 164
Total search depth: 223857
Total times memory accessed: 807437
Total times 24 >= t < 49: 5
Total times 49 >= t < 74: 0
Total times 74 >= t < 99: 0
Total times t == 99: 0
Total time steps survived: 645
Total runtime so far (hours/minutes/seconds): 00:09:38
----------------------------------------
EXPERIMENT COMPLETE ✔.
End Time: 2023-11-28 13:09:14
TOTAL RUNTIME (hours/minutes/seconds): 00:09:38
AVERAGE RUNTIME PER TIME STEP: 0.896 seconds
Average hill visits per time step: 0.043
Average prediction errors per time step: 0.254
Average search depth per time step: 347
Average times memory accessed per time step: 1252
----------------------------------------
